In [42]:
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [43]:
#Twitter credentials for the app
consumer_key = 'Zz8r9wTNoroU47BvboTe4UWeM'
consumer_secret = 'wIUwf7Kfp4f22nJvfOjH1HDCml4BPmAXIVKTOzV3HAt91ZzPWK'
access_key= '1023533182933655552-HtjXvqQt2uCuJc6jHoTzxwOEfSxBNq'
access_secret = 'KguhgyJWHeaVhxgzFPqyyZHBvI09DcahZ5emT7inLKSvx'
 
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [44]:
#file location changed to "data/telemedicine_data_extraction/" for clearer path
tweets_doc = "TSA2_data.csv"

#columns of the csv file
column_names = ['id', 'created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity',
        'lang', 'favorite_count', 'retweet_count', 'original_author', 'possibly_sensitive', 'hashtags',
        'user_mentions', 'place', 'place_coord_boundaries']

In [45]:
#set two date variables for date range
start_date = '2019-5-01'
end_date = '2019-5-31'

In [46]:
#Emojis RE to exclude them
emojis_regex = re.compile(r'[^\u1F600-\u1F6FF\s]+')

#Emoticons
emojis = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)', '<3',
    
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emojis_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", re.UNICODE)

#https://www.vertex42.com/ExcelTips/unicode-symbols.html
#https://en.wikipedia.org/wiki/List_of_emoticons
#http://unicode.org/reports/tr51/

In [91]:
#Function 1 - Preprocessing the tweets
def clean_tweets(tweet):
    #taking stopwords from NLTK - English language
    stop_words = set(stopwords.words('english'))
    #remove emojis from tweet
    tweet = emojis_pattern.sub(r'', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    tweet = re.sub(r'\W',' ', tweet)
    #tokenizing the words
    tokens = word_tokenize(tweet)
    #List to store words of filtered tweet
    filtered_tweet = []
    #removing stopwords by picking each token at a time
    for word in tokens:
        #checking for stopwords, emojis & punctuations
        if word not in stop_words and word not in emojis and word not in string.punctuation:
            filtered_tweet.append(word)
    return ' '.join(filtered_tweet)

In [92]:
#Function 2 - Storing the tweets by writing into a CSV file
def write_tweets(keyword, file):
    df = pd.DataFrame(columns=column_names)
    #page attribute in tweepy.cursor and iteration
    for page in tweepy.Cursor(api.search, q=keyword,
                              count=200, include_rts=False, since=start_date).pages(50):
        for one_tweet_json in page:
            new_entry = []
            one_tweet = one_tweet_json._json
 
            #checking language of the tweet before proceeding
            if one_tweet['lang'] != 'en':
                continue
            #tweet-preprocessor - basic preprocessing
            clean_text = p.clean(one_tweet['text'])
 
            #clean_tweet - my preprocessing function
            filtered_tweet=clean_tweets(clean_text)
 
            #pass textBlob method for sentiment calculations
            blob = TextBlob(filtered_tweet)
            Sentiment = blob.sentiment
 
            #seperate polarity and subjectivity in to two variables
            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity
 
            #new entry append
            new_entry += [one_tweet['id'], one_tweet['created_at'],
                          one_tweet['source'], one_tweet['text'],filtered_tweet, Sentiment,\
                          polarity,subjectivity, one_tweet['lang'],
                          one_tweet['favorite_count'], one_tweet['retweet_count']]
 
            #to append original author of the tweet
            new_entry.append(one_tweet['user']['screen_name'])
 
            try:
                is_sensitive = one_tweet['possibly_sensitive']
            except KeyError:
                is_sensitive = None
            new_entry.append(is_sensitive)
 
            #tagged users
            tagged_users = ", ".join([tagged_user['screen_name'] for \
                                  tagged_user in one_tweet['entities']['user_mentions']])
            new_entry.append(tagged_users)
            
            #hashtags
            hashtags = ", ".join([hashtag['text'] for hashtag in one_tweet['entities']['hashtags']])
            new_entry.append(hashtags)
 
            #get location, if possible
            try:
                location = one_tweet['user']['location']
            except TypeError:
                location = ''
            new_entry.append(location)
 
            try:
                coordinates = [coord for \
                               loc in one_tweet['place']['bounding_box']['coordinates'] for coord in loc]
            except TypeError:
                coordinates = None
            new_entry.append(coordinates)
            
            #Making a single dataframe before appending to the original data frame
            single_tweet_df = pd.DataFrame([new_entry], columns=column_names)
            #Appending to the original dataframe
            df = df.append(single_tweet_df, ignore_index=True)
            csvFile = open(file, 'a' ,encoding='utf-8')
    df.to_csv(csvFile, mode='a', columns=column_names, index=False, encoding="utf-8")

In [93]:
#declare keywords as a query for three categories
tweets_keywords = 'dhoni'

#call main method passing keywords and file path
write_tweets(tweets_keywords, tweets_doc)

In [94]:
df_2 = pd.read_csv("TSA2_data.csv", error_bad_lines = False, lineterminator="\n", encoding = 'latin1')
df_2.head(10)

b'Skipping line 14269: expected 17 fields, saw 20\nSkipping line 14270: expected 17 fields, saw 19\nSkipping line 14271: expected 17 fields, saw 20\nSkipping line 14272: expected 17 fields, saw 21\nSkipping line 14273: expected 17 fields, saw 20\nSkipping line 14279: expected 17 fields, saw 18\nSkipping line 14280: expected 17 fields, saw 18\nSkipping line 14281: expected 17 fields, saw 18\nSkipping line 14284: expected 17 fields, saw 21\nSkipping line 14285: expected 17 fields, saw 28\nSkipping line 14289: expected 17 fields, saw 20\nSkipping line 14299: expected 17 fields, saw 18\nSkipping line 14305: expected 17 fields, saw 19\nSkipping line 14311: expected 17 fields, saw 19\nSkipping line 14317: expected 17 fields, saw 20\nSkipping line 14318: expected 17 fields, saw 19\nSkipping line 14319: expected 17 fields, saw 18\nSkipping line 14326: expected 17 fields, saw 19\nSkipping line 14331: expected 17 fields, saw 21\nSkipping line 14332: expected 17 fields, saw 19\nSkipping line 1433

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries
0,1154207616307871744,Thu Jul 25 01:51:54 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",RT @IndianExpress: #ExpressFrontPage | Faced w...,Faced homebuyers â ire Dhoni â disassociat...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,3,Tweetsandeep,NaN,ExpressFrontPage,IndianExpress,New Delhi,NaN
1,1154207554681147392,Thu Jul 25 01:51:39 +0000 2019,"<a href=""http://twitter.com/download/android"" ...",RT @MohammadKaif: Nuwan Kulasekara retires ! A...,Nuwan Kulasekara retires At one stage number o...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,479,urvishpatel256,NaN,NaN,MohammadKaif,NaN,NaN
2,1154207501392285696,Thu Jul 25 01:51:27 +0000 2019,"<a href=""http://twitter.com/download/android"" ...",RT @imDhoni_fc: There were a total of 523 capt...,There total captains international cricket But...,"Sentiment(polarity=0.26666666666666666, subjec...",0.26666666666666666,0.3833333333333333,en,0,185,manojsabari,NaN,NaN,imDhoni_fc,chennai,NaN
3,1154207425961943040,Thu Jul 25 01:51:09 +0000 2019,"<a href=""http://twitter.com/download/android"" ...",RT @ashoswai: Roads are being prepared for Dho...,Roads prepared Dhoni join BJP election Jharkha...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,21,akhaleems,False,NaN,ashoswai,"Bengaluru, India",NaN
4,1154207415467794432,Thu Jul 25 01:51:06 +0000 2019,"<a href=""http://twitter.com/download/android"" ...","RT @DHONIism: ""Dhoni's Jersey No 7 is availabl...",`` Dhoni 's Jersey No available TEST cricket m...,"Sentiment(polarity=-0.1, subjectivity=0.4)",-0.1,0.4,en,0,287,KareemRio,NaN,NaN,DHONIism,NaN,NaN
5,1154207309582782465,Thu Jul 25 01:50:41 +0000 2019,"<a href=""http://twitter.com/download/android"" ...",RT @ashoswai: Roads are being prepared for Dho...,Roads prepared Dhoni join BJP election Jharkha...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,21,Mudasir62727097,False,NaN,ashoswai,NaN,NaN
6,1154207281329799168,Thu Jul 25 01:50:34 +0000 2019,"<a href=""http://twitter.com/download/android"" ...",RT @jeevanpsharma: Amrapali Diverted Homebuyer...,Amrapali Diverted Homebuyers Money To MS Dhoni...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,52,c_ruhela,False,NaN,jeevanpsharma,NaN,NaN
7,1154207255476297728,Thu Jul 25 01:50:28 +0000 2019,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Hey Dhoni Rohit thanks for the follow!,Hey Dhoni Rohit thanks follow,"Sentiment(polarity=0.2, subjectivity=0.2)",0.2,0.2,en,0,0,usretailproduct,NaN,NaN,NaN,Free shipping on Everything,NaN
8,1154207244856131585,Thu Jul 25 01:50:25 +0000 2019,"<a href=""http://twitter.com/download/android"" ...",RT @CricketNDTV: #MSDhoni retired from Test cr...,retired Test cricket World Test Championship u...,"Sentiment(polarity=-0.2, subjectivity=0.4)",-0.2,0.4,en,0,66,itsDJnawaz,NaN,MSDhoni,CricketNDTV,"Silchar, India",NaN
9,1154207234894643207,Thu Jul 25 01:50:23 +0000 2019,"<a href=""http://twitter.com/download/android"" ...",RT @aarsh_thakur: â¦@msdhoniâ© â¦@harbhajan...,â¦â© â¦â© mr Dhoni check wrong doings With...,"Sentiment(polarity=-0.5, subjectivity=0.9)",-0.5,0.9,en,0,17,c_ruhela,NaN,NaN,"aarsh_thakur, msdhoni, harbhajan_singh",NaN,NaN


In [95]:
print(df_2.iloc[0]['original_text'])

RT @IndianExpress: #ExpressFrontPage | Faced with homebuyersâ ire, Dhoni âdisassociatedâ himself with the Amrapali group in 2016.

https://â¦


In [96]:
print(df_2.iloc[0]['clean_text'])

Faced homebuyers â ire Dhoni â disassociated â Amrapali group https //â¦


In [102]:
a = "RT @IndianExpress: #ExpressFrontPage | Faced with homebuyersâ ire, Dhoni âdisassociatedâ himself with the Amrapali group in 2016."
b = re.sub(r'[^a-zA-Z0-9]',' ', a)
b

'RT  IndianExpress   ExpressFrontPage   Faced with homebuyers    ire  Dhoni    disassociated    himself with the Amrapali group in 2016 '

### Sources: 
1. https://www.kdnuggets.com/2018/03/text-data-preprocessing-walkthrough-python.html
2. https://www.nltk.org/data.html
3. https://www.analyticsvidhya.com/blog/2014/11/text-data-cleaning-steps-python/
4. https://medium.com/@paritosh_30025/natural-language-processing-text-data-vectorization-af2520529cf7
5. https://www.kdnuggets.com/2017/11/framework-approaching-textual-data-tasks.html
6. https://towardsdatascience.com/@jpd.kalpa
7. https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0144296
8. https://cmdlinetips.com/2018/01/how-to-get-unique-values-from-a-column-in-pandas-data-frame/
9. https://www.vertex42.com/ExcelTips/unicode-symbols.html
10. https://en.wikipedia.org/wiki/List_of_emoticons
11. http://unicode.org/reports/tr51/